In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## State feedback control

This example shows the effect of full state feedback.

Given the linear time invariant system:

\begin{cases}
\dot{\textbf{x}}=A\textbf{x}+B\textbf{u} \\
\textbf{y}=C\textbf{x},
\end{cases}

and the control law:

$\textbf{u}=-K\textbf{x}+\textbf{v},$

this example shows the free and forced responses of the close loop system: 


$$
\dot{\textbf{x}}=A\textbf{x}-BK\textbf{x}+B\textbf{v} = (A-BK)\textbf{x}+B\textbf{v}.
$$

### How to use this notebook?
Try to change the values of $K$ or directly set the eigenvalues of $(A-BK)$ and get the corresponding controller gains:
- Create an unstable system and make it stable with full state feedback.
- Create a system with a slow response and make it faster with full state feedback.
- Create a system that is not fully controllable try to change all its eigenvalues in closed loop. Is it possible to achieve that? 
- Create a system that is not fully controllable and unstable and try to make it stable with full state feedback. Can you tell in which cases it is possible to stabilize it with closed-loop control?

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix('0 1 0; 0 0 1; 0 2 -3')
B = numpy.matrix('0; 0; 1')
C = numpy.matrix('1 0 0; 0 1 0; 0 0 1')
X0 = numpy.matrix('2; 2; 2')
K = numpy.matrix([8,14,3])
sol1 = numpy.linalg.eig(A)

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(3,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-2])) 
eig2c.setM(numpy.matrix([[-2],[0]]))
eig3c.setM(numpy.matrix([-2]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Test',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Test',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['Set K', 'Set the eigenvalues'],
    value= 'Set K',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues for the observer
selc = widgets.Dropdown(
    options= ['0 complex eigenvalues', '2 complex eigenvalues'],
    value= '0 complex eigenvalues',
    description='Eigenvalues:',
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulse', 'step', 'sinusoid', 'square wave'],
    value='impulse',
    description='Type of input:',
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description='input u:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.05,
    max=1,
    step=0.05,
    description='Period: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [5]:
# Support functions

def eigen_choice(selc):
    if selc == '0 complex eigenvalues':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eigc = 0
    if selc == '2 complex eigenvalues':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eigc = 2
    return eigc

def method_choice(selm):
    if selm == 'Set K':
        method = 1
        selc.disabled = True
    if selm == 'Set the eigenvalues':
        method = 2
        selc.disabled = False
    return method

In [6]:
def main_callback(Aw, Bw, X0w, K, eig1c, eig2c, eig3c, u, period, selm, selc, selu, DW):
    A, B = Aw, Bw
    sols = numpy.linalg.eig(A)
    eigc = eigen_choice(selc)
    method = method_choice(selm)
    
    if method == 1:
        sol = numpy.linalg.eig(A-B*K)
    if method == 2:
        if eigc == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K) 
        if eigc == 2:
            K = control.acker(A, B, [eig1c[0,0], 
                                      numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        sol = numpy.linalg.eig(A-B*K)
    print('The system\'s eigenvalues are:',round(sols[0][0],4),',',round(sols[0][1],4),'and',round(sols[0][2],4))
    print('The controlled system\'s eigenvalues are:',round(sol[0][0],4),',',round(sol[0][1],4),'and',round(sol[0][2],4))
    
    sys = sss(A,B,C,sym.zeros(3,1))
    sysc = sss(A-B*K, B, numpy.eye(3), numpy.zeros(3).reshape((3,1)))
    T = numpy.linspace(0, 6, 1000)
      
    if selu == 'impulse': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys,T,U,X0w)
        T, youtc, xoutc = control.forced_response(sysc,T,U,X0w)
    if selu == 'step':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U,X0w)
        T, youtc, xoutc = control.forced_response(sysc,T,U,X0w)
    if selu == 'sinusoid':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U,X0w)
        T, youtc, xoutc = control.forced_response(sysc,T,U,X0w)
    if selu == 'square wave':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U,X0w)
        T, youtc, xoutc = control.forced_response(sysc,T,U,X0w)
    
    fig = plt.figure(num='Simulation', figsize=(16,10))
    
    fig.add_subplot(311)
    plt.ylabel('$X_1$ vs $X_{1f}$')
    plt.plot(T,xout[0])
    plt.plot(T,xoutc[0])
    plt.xlabel('time [s]')
    plt.legend(['Open Loop','State Feedback'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(312)
    plt.ylabel('$X_2$ vs $X_{2f}$')
    plt.plot(T,xout[1])
    plt.plot(T,xoutc[1])
    plt.xlabel('time [s]')
    plt.legend(['Open Loop','State Feedback'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(313)
    plt.ylabel('$X_3$ vs $X_{3f}$')
    plt.plot(T,xout[2])
    plt.plot(T,xoutc[2])
    plt.xlabel('time [s]')
    plt.legend(['Open Loop','State Feedback'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          selc, 
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Eigenvalues:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('Dynamics matrix A:',border=3),
                                          Aw,
                                          widgets.Label('Input matrix B:',border=3),
                                          Bw])])
out = widgets.interactive_output(main_callback, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 
                                                 'u':u, 'period':period, 'selm':selm, 'selc':selc, 'selu':selu, 'DW':DW})
out.layout.height = '680px'
display(out, alltogether)

Output(layout=Layout(height='680px'))